# Deploying a TURN Server Using CoTURN

In order to interact with our Tokkio Avatar, we'll need to set up a [media relay service](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/overview/components-and-microservices.html#media-relay-service). This component is responsible for relaying media between the client browser and the Tokkio backend. Especially when the client browser and the Tokkio backend cannot connect directly, they send data to an entity such as a TURN server, which then relays this data to the other device. 

This process ensures that communication can occur even if the direct connections are blocked for reasons such as when they are behind NAT or firewalls. The Tokkio deployment supports the options below as of today.

- COTURN service
- ReverseProxy service
- Twilio

In this notebook, we'll set up a [COTURN](https://github.com/coturn/coturn) service! COTURN is an open-source implementation of TURN and STUN server used for NAT traversal in WebRTC applications. It's expected that the COTURN server is set on a seperate CPU based (non-GPU enabled) virtual machine. Configuring a COTURN service or a TURN server in general on the same instance the Tokkio application is deployed on.

## Import Dependencies

In [12]:
import os

## Setting Up The TURN Server

The TURN server setup consists of installing and configuring the COTURN service. COTURN is an open-source implementation of TURN and STUN server used for NAT traversal in WebRTC applications. The setup involves:

1. Installing the COTURN package
2. Configuring network settings and credentials
3. Setting up the service to run automatically

A TURN server is a requirement in order to ensure the Tokkio services are reachable. We can get started by running the setup script that has been provided by the team for setting up COTURN:

In [1]:
%%bash
# create directory to store scripts in
mkdir -p ../scripts

# store coturn script in newly created directory
cat << 'EOF' > ../scripts/setup-coturn.sh
#!/bin/bash
if [ $1 == 'install' ]; then
  echo "Setup coturn server -- Start"
  DEBIAN_FRONTEND=noninteractive
  apt-get update -y
  apt-get install -y coturn

  if [ -n "$TURNSERVER_PUBLIC_IP" ] && [ -n "${TURNSERVER_PRIVATE_IP}" ]; then
    EXTERNAL_IP="${TURNSERVER_PUBLIC_IP}/${TURNSERVER_PRIVATE_IP}"
  elif [ -z "$TURNSERVER_PUBLIC_IP" ] && [ -n "${TURNSERVER_PRIVATE_IP}" ]; then
    EXTERNAL_IP="${TURNSERVER_PRIVATE_IP}"
  else
    echo "TURNSERVER_PRIVATE_IP variable ${TURNSERVER_PRIVATE_IP} is not set"
  fi

  sed \
    -e "s|^#TURNSERVER_ENABLED=1|TURNSERVER_ENABLED=1|g" \
    -i /etc/default/coturn

  sed \
    -e "/^#\?listening-port=/d" \
    -e "/^#\?listening-ip=/d" \
    -e "/^#\?external-ip=/d" \
    -e "/^#\?relay-ip=/d" \
    -e "/^#\?min-port=/d" \
    -e "/^#\?max-port=/d" \
    -e "/^#\?fingerprint/d" \
    -e "/^#\?realm=/d" \
    -e "/^#\?user=/d" \
    -e "/^#\?log-file=/d" /etc/turnserver.conf \
    -i /etc/turnserver.conf

  cat <<EOT >> /etc/turnserver.conf
listening-port=3478
listening-ip=${TURNSERVER_PRIVATE_IP}
external-ip=${EXTERNAL_IP}
relay-ip=${TURNSERVER_PRIVATE_IP}
min-port=49152
max-port=65535
fingerprint
realm=${TURNSERVER_REALM}
user=${TURNSERVER_USERNAME}:${TURNSERVER_PASSWORD}
log-file=/var/tmp/turn.log
EOT

  systemctl restart coturn

  echo "Setup coturn server -- End"

else
  systemctl stop coturn
  DEBIAN_FRONTEND=noninteractive
  apt-get purge coturn -y
  rm -rf /etc/turnserver.conf
fi
EOF

#### COTURN Script Configuration

Now that we've setup the script, it's time to configure the relevant environment variables. We start by providing the TURN servers realm name, username and password. A realm in COTURN is a logical domain that groups users and credentials, isolating them from other realms and enabling secure, multi-tenant user management on a TURN server. We can configure these values to be whatever we want.

It's also important to carefully check if your Application instance’s IP is behind NAT or not. If your instance is NOT behind a NAT, 
we only set `TURNSERVER_PRIVATE_IP` to the IP address used to login. Otherwise, if the instance is behind a NAT, we set the internal IP as `TURNSERVER_PRIVATE_IP` and set the external (public) IP as `TURNSERVER_PUBLIC_IP`. In other words:

Not Behind NAT:
```
export TURNSERVER_PRIVATE_IP=<IPV4 ADDRESS>      # IP Address of the application instance used to login
```
Behind NAT:
```
export TURNSERVER_PRIVATE_IP=<IPV4 ADDRESS>      # IP Address of the application instance after login
export TURNSERVER_PUBLIC_IP=<IPV4 ADDRESS>      # IP Address of the application instance used to login
```

#### Verifying If Instances Are Behind A NAT

To check if your Linux VM is behind a NAT, you can compare private and public IP Addresses. If your VM's local IP is in a private range (such as 10.x.x.x, 192.168.x.x, or 172.16.x.x–172.31.x.x) and the public IP is different, your VM is behind a NAT:

In [2]:
%%bash
# command to print out private IP addresses
echo "Private IP Addresses: $(hostname -I)"

# command to print out public IP address
echo "Public IP Address: $(curl ifconfig.me)"

Private IP Addresses: 172.31.9.54 172.17.0.1 192.168.35.0 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0    409      0 --:--:-- --:--:-- --:--:--   411


Public IP Address: 35.175.131.103


#### Configure Script Env's

From the output above, we've verified our machine is behind a NAT, so we configure the `TURNSERVER_PRIVATE_IP` and `TURNSERVER_PUBLIC_IP` variables:

In [8]:
os.environ["TURNSERVER_REALM"] = "mydummy.org"  # Realm name
os.environ["TURNSERVER_USERNAME"] = "foo"  # Username
os.environ["TURNSERVER_PASSWORD"] = "bar"  # Password
os.environ["TURNSERVER_PRIVATE_IP"] = "172.31.9.54" # use any of the private IP addresses returned
os.environ["TURNSERVER_PUBLIC_IP"] = "35.175.131.103" # use the public IP address returned

#### Install COTURN

Once we've configured the appropriate environment variables, we can run the following code block to execute the script and bring up the COTURN server

In [ ]:
%%bash
chmod u+x ../scripts/setup-coturn.sh
sudo -E ../scripts/setup-coturn.sh install

#### Verify COTURN Server Is Running

We can use the command below to verify the COTURN Service is up and running - if we see the `Active: active (running)` status, the service is running as expected.

In [5]:
%%bash
sudo systemctl status coturn.service

● coturn.service - coTURN STUN/TURN Server
     Loaded: loaded (/lib/systemd/system/coturn.service; enabled; vendor preset: enabled)
     Active: active (running) since Tue 2025-05-06 17:53:50 UTC; 53min ago
       Docs: man:coturn(1)
             man:turnadmin(1)
             man:turnserver(1)
   Main PID: 791669 (turnserver)
      Tasks: 75 (limit: 457992)
     Memory: 18.3M
        CPU: 3.533s
     CGroup: /system.slice/coturn.service
             └─791669 /usr/bin/turnserver -c /etc/turnserver.conf --pidfile=

May 06 17:53:50 ip-172-31-9-54 turnserver[791669]: 0: : IO method (auth thread): epoll (with changelist)
May 06 17:53:50 ip-172-31-9-54 turnserver[791669]: 0: : IO method (auth thread): epoll (with changelist)
May 06 17:53:50 ip-172-31-9-54 turnserver[791669]: 0: : IO method (auth thread): epoll (with changelist)
May 06 17:53:50 ip-172-31-9-54 turnserver[791669]: 0: : IO method (auth thread): epoll (with changelist)
May 06 17:53:50 ip-172-31-9-54 turnserver[791669]: 0: : IO m

#### Verify TURN Server IP

The command below is also useful for verifying the IP address our TURN Server is listening on. In this example, the output is returned, which means the public IP `35.175.131.103` is mapping to our machines internal IP address on `172.31.9.54`:
```
external-ip=35.175.131.103/172.31.9.54
```

This will be useful later when configuring our Tokkio chart!

In [6]:
%%bash
sudo grep -E 'external-ip|listening-ip' /etc/turnserver.conf

# The "external-ip" value, if not empty, is returned in XOR-RELAYED-ADDRESS field.
listening-ip=172.31.9.54
external-ip=35.175.131.103/172.31.9.54


#### Testing TURN Server Connection

Using Command-Line Tools like `turnutils`, we can verify the connection to our TURN server that was set up. CoTURN includes utilities for testing. The most useful is `turnutils_uclient`, which simulates a TURN client. 

We'll use the same username and password that was used to configure the TURN server. Additionally, since this machine that is being used is currently behind a NAT, we'll leverage our machines private IP address to test the connection. Outside of this machine, the public IP address will need to be leveraged to facilitate communication with the TURN server.

In [11]:
%%bash
turnutils_uclient -v -u $TURNSERVER_USERNAME -w $TURNSERVER_PASSWORD $TURNSERVER_PRIVATE_IP

0: : IPv4. Connected from: 172.31.9.54:57773
0: : IPv4. Connected from: 172.31.9.54:57773
0: : IPv4. Connected to: 172.31.9.54:3478
0: : IPv4. Connected to: 172.31.9.54:3478
0: : allocate sent
0: : allocate sent
0: : allocate response received: 
0: : allocate response received: 
0: : allocate sent
0: : allocate sent
0: : allocate response received: 
0: : allocate response received: 
0: : success
0: : success
0: : IPv4. Received relay addr: 35.175.131.103:51912
0: : IPv4. Received relay addr: 35.175.131.103:51912
0: : clnet_allocate: rtv=13898181692145200660
0: : clnet_allocate: rtv=13898181692145200660
0: : refresh sent
0: : refresh sent
0: : refresh response received: 
0: : refresh response received: 
0: : success
0: : success
0: : IPv4. Connected from: 172.31.9.54:60318
0: : IPv4. Connected from: 172.31.9.54:60318
0: : IPv4. Connected to: 172.31.9.54:3478
0: : IPv4. Connected to: 172.31.9.54:3478
0: : IPv4. Connected from: 172.31.9.54:59793
0: : IPv4. Connected from: 172.31.9.54:5979

CalledProcessError: Command 'b'turnutils_uclient -v -u $TURNSERVER_USERNAME -w $TURNSERVER_PASSWORD $TURNSERVER_PRIVATE_IP\n'' returned non-zero exit status 255.

#### Next Steps
The output above indicates that the CoTURN server is mostly working as expected for basic TURN functionality, but there is one notable exception: a 403 (Forbidden IP) error during a channel bind attempt. To be clear if the output mostly completes above, that means basic TURN functionality is working - this is sufficient for our Tokkio application! 

Below we explain the output shown above, but if `Allocation` & `Refresh` functionality is working we're good to go with the `tokkio_app_deploy.ipynb` notebook! Please make sure the TURNa server you have installed on your CPU based VM is accessible and reachable from the instance you're deploying your Tokkio application unto - otherwise, you may run into issues connecting with your Avatar.

#### Analyzing The Output Above 
Let's analyze what this means:

#### What's Working
- Allocation Succeeded: The server responded successfully to your allocation requests:
```
text
0: : allocate sent
0: : allocate response received:
0: : success
0: : IPv4. Received relay addr: 35.175.131.103:58818
```
This means you received a relay address, which is the core TURN functionality.

- Refresh Succeeded: The server also accepted your refresh requests:

```
text
0: : refresh sent
0: : refresh response received:
0: : success
```

#### What's Not Working
403 Forbidden IP on Channel Bind:

```
text
0: : channel bind sent
0: : cb response received:
0: : channel bind: error 403 (Forbidden IP)
```
This error means the server refused the channel bind request, usually because the IP address you're trying to bind is not allowed according to the server's configuration or security restrictions.

#### Summary Table

| Functionality   | Status | Notes                   |
|-----------------|:------:|-------------------------|
| Allocation      |   ✅   | Relay address assigned  |
| Refresh         |   ✅   | Refresh successful      |
| Channel Bind    |   ❌   | 403 Forbidden IP error  |

#### What Should You Do Next?
- Basic TURN is working: For most WebRTC use cases, allocation and refresh are the critical parts. Channel bind is less commonly used and may not be required for your application. This is sufficient for our Tokkio application.
- If you need channel bind functionality, check your CoTURN configuration for any IP restrictions or firewall rules that might be blocking the bind request.